In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.ticker as ticker
import IPython.display
import pygame

NOTEBOOK_ID = "COLLISION_BASIS"
OUTPUT_PATH = f"out/{NOTEBOOK_ID}/"

if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

plt.rcParams['font.family'] = ["D2Coding"]
pygame.init()


# 점과 물체의 충돌 판별

어떤 점 $P(x,y)$가 물체와 충돌을 한다고 하면 최소한 그 물체 내부 혹은 그 경계에 있다는 것이다.

## 점과 원의 충돌 판별

점 $P$가 임의의 중심을 $C(C_x, C_y)$으로 하고 반지름을 $r$로 하는 원 $O$와 충돌한다고 하면 다음이 성립해야 한다.

> $$\overline{PC} \leq r$$

이는 점과 점 사이의 거리 공식에 따라 다음과 같이 쓸 수 있다.

> $$(C_x-x)^2+(C_y-y)^2 \leq r^2$$

## 점과 변이 축과 평행한 사각형의 충돌 판별

점 $P$가 각 꼭짓점이 $(x_1, y_1),\,(x_2, y_1),\,(x_1,y_2),\,(x_2,y_2)\quad(x_1 < x_2, \,y_1 < y_2)$인 사각형과 충돌한다고 하면 다음이 성립해야 한다.

> $$x_1\leq x\leq x_2\quad \text{and}\quad y_1 \leq y \leq y_2$$

## 점과 삼각형의 충돌

점 $P$가 각 꼭짓점이 $A(x_1, y_1),\,B(x_2, y_2),\,C(x_3,y_3)$인 삼각형 $\triangle{ABC}$ 내부에 존재한다고 하면 다음이 성립한다.

> $$V_0 = \vec{AP}\times\vec{AB},\,V_1=\vec{BP}\times\vec{BC},V_2=\vec{CP}\times\vec{CA}$$
> $$V_0, V_1, V_2 \leq 0\quad \text{or} \quad V_0, V_1, V_2 \geq 0$$

다만 외적을 이용하여 상대적으로 느리다.

이를 통해 모든 다각형은 삼각형으로 나눌 수 있으므로 일반화하여 다각형 내부에 점이 있는지 판별하는 조건을 만들 수 있다.